In [150]:
import cv2
from PIL import Image
import numpy as np
import pandas as pd
import os
from glob import glob
from tqdm import tqdm
import imageio
import pydicom as dicom
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import f1_score, jaccard_score, precision_score, recall_score, accuracy_score
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input, Flatten
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.metrics import Recall, Precision, IoU
from albumentations import HorizontalFlip, VerticalFlip, Rotate

In [3]:
H = 512
W = 512

In [5]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [38]:
def load_data(dataset_path, split=0.2):
    images = sorted(glob(f"{dataset_path}\\*\\image\\*.png"))
    masks = sorted(glob(f"{dataset_path}\\*\\mask\\*.png"))
    print(len(images), len(masks))

    split_size = int(len(images) * split)
    X_train, X_val = train_test_split(images, test_size=split_size, random_state=42)
    y_train, y_val = train_test_split(masks, test_size=split_size, random_state=42)

    print("Training : ", len(X_train), len(y_train))
    print("Validation : ", len(X_val), len(y_val))

    return (X_train, y_train), (X_val, y_val)

In [54]:
def augmentation_data(images, masks, save_path, augment=True):

    H = 512
    W = 512

    for idx, (x, y) in tqdm(enumerate(zip(images, masks)), total=len(images)):
        #print(x)
        #print(y)

        dir_name = x.split("\\")[-3]
        name = x.split("\\")[-1].split(".")[0]
        full_name = dir_name + "_" + name
        #print(dir_name)
        #print(name)
        #print(full_name)

        x = cv2.imread(x, cv2.IMREAD_COLOR)
        y = cv2.imread(y, cv2.IMREAD_COLOR)

        if augment == True:
            aug = HorizontalFlip(p=1.0)
            augmented = aug(image=x, mask=y)
            x1 = augmented["image"]
            y1 = augmented["mask"]

            aug = VerticalFlip(p=1.0)
            augmented = aug(image=x, mask=y)
            x2 = augmented["image"]
            y2 = augmented["mask"]

            aug = Rotate(limit=45, p=1.0)
            augmented = aug(image=x, mask=y)
            x3 = augmented["image"]
            y3 = augmented["mask"]

            X = [x, x1, x2, x3]
            y = [y, y1, y2, y3]

        else:
            X = [x]
            y = [y]

        index = 0
        for i, m in zip(X, y):
            i = cv2.resize(i, (W, H))
            m = cv2.resize(m, (W, H))
            m = m / 255.0
            m = (m > 0.5) * 255

            if len(X) == 1:
                tmp_image_name = f"{full_name}.jpg"
                tmp_mask_name = f"{full_name}.jpg"

            else:
                tmp_image_name = f"{full_name}_{index}.jpg"
                tmp_mask_name = f"{full_name}_{index}.jpg"

            image_path = os.path.join(save_path, "images\\", tmp_image_name)
            mask_path = os.path.join(save_path, "masks\\", tmp_mask_name)

            cv2.imwrite(image_path, i)
            cv2.imwrite(mask_path, m)
                
            index += 1

In [56]:
if __name__=="__main__":
    dataset_path = r"E:\python\segmentation\Computer Vision\Data\CTarchive\data/"
    dataset = os.path.join(dataset_path, "train")
    (X_train, y_train), (X_val, y_val) = load_data(dataset, split=0.2)
    print("train : ", len(X_train))
    print("validataion : ", len(X_val))

    create_dir(dataset_path + "new_data/train/images")
    create_dir(dataset_path + "new_data/train/masks")
    create_dir(dataset_path + "new_data/val/images")
    create_dir(dataset_path + "new_data/val/masks")

    save_path_train = r"E:\python\segmentation\Computer Vision\Data\CTarchive\data\new_data\train"
    save_path_test = r"E:\python\segmentation\Computer Vision\Data\CTarchive\data\new_data\val"

    augmentation_data(X_train, y_train, save_path_train, augment=True)
    augmentation_data(X_val, y_val, save_path_test, augment=False)

2532 2532
Training :  2026 2026
Validation :  506 506
train :  2026
validataion :  506


100%|████████████████████████████████████████████████████████████████████████████████| 506/506 [00:06<00:00, 73.36it/s]


In [58]:
def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

In [60]:
def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

In [62]:
def decoder_block(input, skip_feature, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_feature])
    x = conv_block(x, num_filters)
    return x

In [64]:
def build_unet(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b = conv_block(p4, 1024)

    d1 = decoder_block(b, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="Unet")

    return model

In [66]:
if __name__=="__main__":
    input_shape = (512, 512, 3)
    model = build_unet(input_shape)
    model.summary()



Model: "Unet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 512, 512, 64)         1792      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 512, 512, 64)         256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 512, 512, 64)         0         ['batch_normalization[0][

In [68]:
def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x

    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

In [170]:
smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = Flatten()(y_true)
    y_pred = Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

In [186]:
def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [117]:
def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

In [119]:
def load_dataset(path):
    x = sorted(glob(os.path.join(path, "images", "*.jpg")))
    y = sorted(glob(os.path.join(path, "masks", "*.jpg")))
    return x, y

In [121]:
def read_images(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    #x = cv2.resize(x, (W, H))
    x = x / 255.0
    x = x.astype(np.float32)
    return x

In [123]:
def read_masks(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    #x = cv2.resize(x, (W, H))
    x = x / 255.0
    x = x > 0.5
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)
    return x

In [125]:
def tf_parse(x, y):
    def parse(x, y):
        x = read_images(x)
        y = read_masks(y)
        return x, y

    x, y = tf.numpy_function(parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

In [127]:
def tf_dataset(x, y, batch=0):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    #dataset = dataset.repeat()
    dataset = dataset.prefetch(10)
    return dataset

In [138]:
if __name__=="__main__":
    np.random.seed(42)
    tf.random.set_seed(42)

    save_model_path = r"E:\python\segmentation\Computer Vision\Data\CTarchive\data"
    create_dir(save_model_path + "/files")

    batch_size = 2
    learning_rate = 1e-4
    num_epochs = 10
    model_path = os.path.join(save_model_path , "files" , "CT_Scan.h5")
    csv_path = os.path.join(save_model_path , "files" , "CT_scan.csv")

    data = r"E:\python\segmentation\Computer Vision\Data\CTarchive\data\new_data"

    train_path = os.path.join(data, "train")
    
    validation_path = os.path.join(data, "val")

    X_train1, y_train1 = load_dataset(train_path)
    X_train1, y_train1 = shuffling(X_train1, y_train1)
    X_val1, y_val1 = load_dataset(validation_path)

    print(f"Training: {len(X_train1)} - {len(y_train1)}")
    print(f"Valid: {len(X_val1)} - {len(y_val1)}")

    train_dataset = tf_dataset(X_train, y_train, batch=batch_size)
    val_dataset = tf_dataset(X_val, y_val, batch=batch_size)

    model = build_unet((H, W, 3))
    metrics = [dice_coef, iou, Recall(), Precision()]
    model.compile(loss=dice_loos, optimizer=Adam(learning_rate=learning_rate), metrics=metrics)

    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=10, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path),
        #TensorBoard(),
        EarlyStopping(monitor="val_loss", patience=50, restore_best_weights=False),
    ]

    model.fit(
    train_dataset,
    epochs=1, #num_epochs,
    validation_data=val_dataset,
    callbacks=callbacks,
    shuffle=False
)

Training: 8108 - 8108
Valid: 507 - 507
1013/1013 [==============================] - ETA: 0s - loss: 0.7065 - dice_coef: 0.2935 - iou: 0.2101 - recall_8: 0.7865 - precision_8: 0.2302
Epoch 1: val_loss improved from inf to 0.69781, saving model to E:\python\segmentation\Computer Vision\Data\CTarchive\data\files\CT_Scan.h5


C:\Users\hamid\anaconda3\envs\MachineLearning\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1013/1013 [==============================] - 9850s 10s/step - loss: 0.7065 - dice_coef: 0.2935 - iou: 0.2101 - recall_8: 0.7865 - precision_8: 0.2302 - val_loss: 0.6978 - val_dice_coef: 0.3022 - val_iou: 0.2065 - val_recall_8: 0.9837 - val_precision_8: 0.2505 - lr: 1.0000e-04


In [142]:
model.save("Unet_segmentation_on_ct_scan.h5")

In [144]:
def save_results(image, mask, y_pred, save_image_path):
    line = np.ones((H, 10, 3)) * 255
    
    mask = np.expand_dims(mask, axis=-1)
    mask = np.concatenate([mask, mask, mask], axis=-1)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1)
    y_pred = y_pred * 255

    #print(image.shape, mask.shape, y_pred.shape)
    cat_images = np.concatenate([image, line, mask, y_pred], axis=1)
    cv2.imwrite(save_image_path, cat_images)

In [215]:
if __name__ == "__main__":
    np.random.seed(42)
    tf.random.set_seed(42)

    path = r"E:\python\segmentation\Computer Vision\Data\CTarchive\data\files"
    create_dir(path + "result")
    
    my_model = r"E:\python\segmentation\Computer Vision\Data\CTarchive\data\files\CT_Scan.h5"

    with CustomObjectScope({"dice_coef":dice_coef, "dice_loss":dice_loos}):
        loaded_model = tf.keras.models.load_model(my_model, compile=False)

    #loaded_model.summary()
    test_path = r"E:\python\segmentation\Computer Vision\Data\CTarchive\data\new_data"
    X_test = sorted(glob(os.path.join(test_path, "val", "images", "*")))
    y_test = sorted(glob(os.path.join(test_path, "val", "masks", "*")))
    print(f"Test: {len(X_test)} - {len(y_test)}")

    #(X_train, y_train_mask), (X_validation, y_validation_mask), (X_test, y_mask_test) = load_dataset(dataset)

    score = []
    for x, y in tqdm(zip(X_test, y_test), total=len(X_test)):
        name = x.split("\\")[-1].split(".")[0]
        print("name: ", name)

        image = cv2.imread(x, cv2.IMREAD_COLOR)
        #image = cv2.resize(image, (W, H))
        x = image / 255.0
        #x = x.astype(np.float32)
        x = np.expand_dims(x, axis=0)
    
        mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
        #mask = cv2.resize(mask, (W, H))
        y = mask / 255.0
        y = y > 0.5
        y =y.astype(np.int32)
                                
        y_pred = loaded_model.predict(x)[0]
        y_pred = np.squeeze(y_pred, axis=-1)
        #print(y_pred.shape)
        y_pred = y_pred >= 0.5
        y_pred = y_pred.astype(np.int32)

        save_image_path = r"E:\python\segmentation\Computer Vision\Data\CTarchive\data\filesresult/"
        save_image = f"{name}.png"
        save_image_path = os.path.join(save_image_path, save_image)
        save_results(image, mask, y_pred, save_image_path)

        y = y.Flatten()
        y_pred = y_pred.Flatten()

        acc_value = accuracy_score(y, y_pred)
        f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary", zero_division=1)
        jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary", zero_division=1)
        recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary", zero_division=1)
        precision_value = precision_score(y, y_pred, labels=[0, 1], average="binary", zero_division=1)

        score.append([name, acc_value ,f1_value, jac_value, recall_value, precision_value])
    print(score)
    score = [s[1:] for s in score]
    score = np.mean(score, axis=0)
    print(f"Accuracy: {score[0]:0.5f}")
    print(f"F1: {score[1]:0.5f}")
    print(f"Jaccard: {score[2]:0.5f}")
    print(f"Recall: {score[3]:0.5f}")
    print(f"Precision: {score[4]:0.5f}")

    df = pf.DataFrame(score, columns=["Image", "Accuracy", "F1", "Jaccard", "Recall", "Precision"])
    df.to_csv("files/score.csv")

Test: 507 - 507


  0%|                                                                                          | 0/507 [00:00<?, ?it/s]

name:  1-090
1/1 [==============================] - 4s 4s/step


  0%|                                                                                          | 0/507 [00:04<?, ?it/s]


In [227]:
test = r"E:\python\segmentation\Computer Vision\Data\CTarchive\data"
if __name__ == "__main__":
    np.random.seed(42)
    tf.random.set_seed(42)

    create_dir(path + "result")
    my_model = r"E:\python\segmentation\Computer Vision\Data\CTarchive\data\files\CT_Scan.h5"

    with CustomObjectScope({"dice_coef":dice_coef, "dice_loss":dice_loss}):
        loaded_model = tf.keras.models.load_model(my_model, compile=False)

    X_test = glob(test + "\\test\\*\\*\\*.dcm")
    print(f"Test: {len(X_test)}")

    for x in tqdm(X_test):
        direc_name = x.split("\\")[-3]
        test_name = direc_name + "_" + x.split("\\")[-1].split(".")[0]

        image = dicom.dcmread(x).pixel_array
        image = np.expand_dims(image, axis=-1)
        print(image.shape)
        #print(np.max(image))

        image = image / np.max(image) * 255.0
        x = image / 255.0
        x = np.concatenate([x, x, x], axis=-1)
        x = np.expand_dims(x, axis=0)

        mask = loaded_model.predict(x)[0]
        mask = mask > 0.5
        mask = mask.astype(np.int32)
        mask = mask * 255

        cat_images = np.concatenate([image, mask], axis=1)
        test_save = r"E:\python\segmentation\Computer Vision\Data\CTarchive\data"
        test_save = os.path.join(test_save, "files")
        cv2.imwrite(test_save + f"\\{test_name}.png", cat_images)


        break

Test: 832


  0%|                                                                                          | 0/832 [00:00<?, ?it/s]

(512, 512, 1)
1/1 [==============================] - 4s 4s/step


  0%|                                                                                          | 0/832 [00:04<?, ?it/s]
